# Library Import

In [ ]:
!pip install deepface tf-keras matplotlib

## Library Install

In [2]:
from deepface import DeepFace
from retinaface import RetinaFace

25-02-17 06:40:46 - Directory /root/.deepface has been created
25-02-17 06:40:46 - Directory /root/.deepface/weights has been created


In [3]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import glob

In [35]:
import json

Use Google Colab

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')
ROOT_DIR = "/content/gdrive/MyDrive/Colab Notebooks/FR_Project/"

Mounted at /content/gdrive


# Main Logic

## File structure

In [5]:
from pathlib import Path
import shutil


def mkdir(p):
  Path(p).mkdir(parents=True, exist_ok=True)


In [48]:
SOURCE_DIR = f"{ROOT_DIR}Raw_Image_Source"
PROCESSED_IMAGE_TXT = f"Processed.txt"

In [86]:
DRIVE_OUTPUT = False

if DRIVE_OUTPUT:
  FACES_INDEX = f"{ROOT_DIR}Faces_Index"
  FACES_DIR = f"{ROOT_DIR}Faces"
  NON_FACES_DIR = f"{ROOT_DIR}Non_Faces"
  SCORE_INDEX_LIST_PATH = f"{FACES_INDEX}/score_index_list.json"
else:
  FACES_INDEX = f"Faces_Index"
  FACES_DIR = f"Faces"
  NON_FACES_DIR = f"Non_Faces"
  SCORE_INDEX_LIST_PATH = f"{FACES_INDEX}/score_index_list.json"

# TEMP_DIR = "Temp"
mkdir(FACES_INDEX)
mkdir(FACES_DIR)
mkdir(NON_FACES_DIR)
# mkdir(TEMP_DIR)

def write_processed(img_path):
  with open(PROCESSED_IMAGE_TXT, "a+") as txt_file:
    txt_file.writelines(img_path + "\n")


Init JSON score index list

In [137]:
score_index_list = {}

if not Path(SCORE_INDEX_LIST_PATH).is_file():
  with open(SCORE_INDEX_LIST_PATH, 'w') as f:
        json.dump({}, f)

with open(SCORE_INDEX_LIST_PATH, 'r+') as f:
    score_index_list = json.load(f)

In [149]:
img = Image.new('RGB',(480,640),"rgb(255,255,255)")
img.save(f"{FACES_INDEX}/dummy.jpg")

## **Function:** Image view with PIL

drawImageWithPlot( image_Path, param, save_crop=False, *, show_img=False )

---

> param: {
  "face_area_1": {
    facial_area: [x, y, w, h],
    ...
  },
  "face_area_2": {
    facial_area: [x, y, w, h],
    ...
  }, ...
}

In [8]:
def plt_show_img(imgs: list) -> None:
  imgs_count = len(imgs)
  for i in range(imgs_count):
    ax = plt.subplot(1, imgs_count, i+1)
    ax.imshow(imgs[i])
    ax.set_xticks([])
    ax.set_yticks([])
  plt.tight_layout(h_pad=3)
  plt.show()

def drawImageWithPlot(img_path, param, save_crop=False, *, show_img=False):
  im = Image.open(img_path)
  crop_array = []
  crop_len = len(crop_array)
  crop_count = 0

  draw_img_grid = []

  for key in param:
    rect = param[key]['facial_area']
    x, y, w, h = rect

    if save_crop > 0:
      im_crop = im.crop((x-100, y-100, w+100, h+100))
      save_path = crop_array[crop_count] if crop_count < crop_len else f"{FACES_DIR}/saved_crop_{crop_count}.jpg"
      im_crop.save(save_path)
      crop_count = crop_count + 1

    border_width = int(min(w-x, h-y) / 15)
    border_width = max(border_width, 10)
    border_width = min(border_width, 20)

    draw_img_grid.append((x, y, w, h, border_width))
    # draw = ImageDraw.Draw(im)
    # draw.rectangle((x, y, w, h), outline="red", width=border_width)

  if show_img:
    draw = ImageDraw.Draw(im)
    for x, y, w, h, border_width in draw_img_grid:
      draw.rectangle((x, y, w, h), outline="red", width=border_width)
    plt_show_img([im])

## Function: Save Crop Image as Face_index

In [112]:
# Model List
# models_name = ["VGG-Face", "Facenet", "Facenet512", "OpenFace",
#         "DeepID", "ArcFace", "SFace"]
models_name = ["Facenet512", "OpenFace", "DeepID", "SFace"]

In [54]:
def gen_next_img_full_path():
  i = 1
  while True:
    if f"{FACES_INDEX}/person{i}.jpg" not in score_index_list:
      return f"{FACES_INDEX}/person{i}.jpg"
    i = i + 1

In [166]:
# True: replace
# False: index no change
def check_new_index_file(img_name, score) -> bool:
  if img_name not in score_index_list:
    return True
  curr_score = score_index_list[img_name]
  return curr_score < score

def check_face_index(img_path, param):
  im = Image.open(img_path)
  found_person = [f'{FACES_INDEX}/dummy.jpg'] # two person will not appear in same image

  for key in param:
    score = param[key]['score']
    rect = param[key]['facial_area']
    x, y, w, h = rect
    total_width = w - x
    total_height = h - y

    im_crop = im.crop((x - total_width*0.3, y - total_height*0.4, w + total_width*0.3, h + total_height*0.4))

    # filter too small images
    if w - x <= 30 or h - y <= 30:
      print(f"image too small: {rect}")
      continue

    # filter score too low?
    if score < 0.96:
      print(f"score too low: {score}")
      continue

    save_path = f"Crop_Image_Temp.jpg"
    im_crop.save(save_path)

    match_res = []
    match_model = []
    for x in models_name:
      result = DeepFace.find(img_path=save_path, db_path=FACES_INDEX, model_name=x, enforce_detection=False, silent=True)
      for res in result:
        if res.empty: continue
        r = res["identity"][0]
        if r not in match_res:
          match_res.append(r)
        match_model.append(x)

    print(f"Image Score: {score}")
    print(f"Matched: {match_res}")
    print(f"Reported by: {match_model}")
    found_img_full_path = ''
    if match_res and match_res[0] not in found_person:
      found_img_full_path = match_res[0]
    else:
      found_img_full_path = gen_next_img_full_path()

    print(f"found_img_full_path: {found_img_full_path}")
    if check_new_index_file(found_img_full_path, score):

      if found_img_full_path in score_index_list:
        print(f"Replacing {found_img_full_path} from {score_index_list[found_img_full_path]} -> {score}")
      else:
        print(f"New Person: {found_img_full_path} with score {score}")
      shutil.move(save_path, f"{found_img_full_path}")
      score_index_list[found_img_full_path] = score
      with open(SCORE_INDEX_LIST_PATH, 'w') as f:
        json.dump(score_index_list, f)

    found_person.append(found_img_full_path)
    print()


## Image Analyze

In [ ]:
image_list = glob.glob(f"{SOURCE_DIR}/*.jpg")

Path(PROCESSED_IMAGE_TXT).touch(exist_ok=True)
with open(PROCESSED_IMAGE_TXT, "r") as txt_file:
  lines = txt_file.readlines()

for x in lines:
  image_list.remove(x.replace("\n", ''))

for img_path in image_list:
  resp = RetinaFace.detect_faces(img_path)

  write_processed(img_path)
  # drawImageWithPlot(img_path, resp, True, False)
  if resp:
    check_face_index(img_path, resp)
    break

  target_path = img_path.replace(f"{SOURCE_DIR}/", "")
  shutil.copyfile(img_path, f"{NON_FACES_DIR}/{target_path}")

In [ ]:
drawImageWithPlot(img_path, resp, False, show_img=True)

Extract all faces from 1 image

In [ ]:
# analyze one image
img_path = "test2.jpg"
resp = RetinaFace.detect_faces(img_path)

In [ ]:
# output image
drawImageWithPlot(img_path, resp, True)

In [ ]:
result = DeepFace.find(img_path="saved_crop_0.jpg", db_path="find_image", model_name="VGG-Face", enforce_detection=False, silent=True)

In [ ]:
print(result)

# Testing Area

In [ ]:
face_objs = DeepFace.extract_faces(
  img_path = "test.jpg",
  detector_backend = "dlib",
  align = True,
)
face_objs

In [ ]:
img = cv2.imread('test.jpg')
plt.imshow(img[:, :, ::-1])

In [ ]:
type(face_objs)